In [29]:
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import seaborn as sns


---
# Collecting data
Our data is about Rolex watches which are sold on `chrono24.com`.

Yes, He allows us to freely use the dataset.

He collected this data by scraping with Selenium

---
# Exploring data

In [30]:
rolex_df = pd.read_csv('rolex_scaper_clean.csv')

#### Number of rows and columns

In [31]:
num_rows,num_cols = rolex_df.shape
rolex_df.shape

(87117, 12)

#### Meaning of each row
We can see that each row has information like model, price, ... about the watch.

In [32]:
rolex_df.head()

,model,reference number,price,aditional shipping price,ad name,movement,case material,case diameter,year of production,condition,scope of delivery,location
0,Rolex Lady-Datejust,179161,9080.0,140.0,Steel Rose Gold Black Roman Dial Ladies Watch ...,Automatic,Steel,26 mm,2014.0,Very good,"Original box, original papers","United States of America, Georgia"
1,Rolex Chronograph,2917,16202.0,216.0,REF. 2917,Manual winding,Steel,33 mm,1934.0,Very good,"Original box, no original papers","Italy, Roma"
2,Rolex Daytona,116519G,41567.0,0.0,保証書付き ROLEX ロレックス デイトナ コスモグラフ K18WG 8Pダイヤ ランダム...,Automatic,White gold,39 mm,NaN,Fair,"Original papers, no original box","Japan, Nagoya City"
3,Rolex Submariner Date,116613,19795.0,235.0,New Submariner 116613 Yellow Steel Gold Cerami...,Automatic,Steel,40 mm,2020.0,Unworn,"Original box, original papers","United States of America, Florida, Miami"
4,Rolex Submariner Date,16610,10674.0,145.0,1990s ROLEX SUBMARINER 16610 Tritium vintage G...,Automatic,NaN,40 mm,1990.0,Good,"Original box, original papers","Japan, Ehime yawatahama"


#### Are there duplicated rows ?

In [33]:
rolex_df.duplicated(keep='first').sum()

21898

Drop duplicated rows

In [34]:
rolex_df.drop_duplicates(inplace=True)

#### Meaning of each columns ?

model: the watch model name

reference number: the number to identify the watch model as a whole

price: the price on the listing (price made by the reseller)

additional shipping price: the price for shipping (0 = free shipping)

ad name: the name of the listing on the site

movement: the engine of a watch to make the watch and its functions operate

case material: the material of the external watch case

case diameter: the dimensions of the watch

year of production: the year when Rolex fabricate that particular watch

condition: the general condition of the watch

scope of delivery: the additional things like warranty, or box that could come with the watch

location: the location of the reseller


#### Type of each colum 
Luckily, all columns are in their right data types except for year of production. It should be categorical (object) instead of float64.

In [35]:
rolex_df.dtypes 

model                        object
reference number             object
price                       float64
aditional shipping price    float64
ad name                      object
movement                     object
case material                object
case diameter                object
year of production          float64
condition                    object
scope of delivery            object
location                     object
dtype: object

Change data types of year of production column

In [36]:
rolex_df['year of production'] = rolex_df['year of production'].astype('object')
rolex_df.dtypes

model                        object
reference number             object
price                       float64
aditional shipping price    float64
ad name                      object
movement                     object
case material                object
case diameter                object
year of production           object
condition                    object
scope of delivery            object
location                     object
dtype: object

#### With each numerical column, how are values distributed?

In [37]:
numerical_cols = rolex_df.select_dtypes(include=np.number).copy()
numerical_cols.head()

,price,aditional shipping price
0,9080.0,140.0
1,16202.0,216.0
2,41567.0,0.0
3,19795.0,235.0
4,10674.0,145.0


Proportion of missing values

In [38]:
num_missing_percentages = (numerical_cols.isna()).sum() / rolex_df.shape[0] * 100
num_missing_percentages

price                       6.271179
aditional shipping price    0.000000
dtype: float64

Describe

In [39]:
numerical_cols.describe()

,price,aditional shipping price
count,6.112900e+04,65219.000000
mean,2.276485e+04,134.830234
std,3.059692e+04,2580.042710
min,3.150000e+02,0.000000
25%,9.157000e+03,31.000000
50%,1.500000e+04,99.000000
75%,2.483300e+04,160.000000
max,1.152102e+06,656389.000000


#### With each categorical column, how are values distributed?

In [40]:
categorical_cols = rolex_df.select_dtypes(exclude=np.number).copy()
categorical_cols.head()

,model,reference number,ad name,movement,case material,case diameter,year of production,condition,scope of delivery,location
0,Rolex Lady-Datejust,179161,Steel Rose Gold Black Roman Dial Ladies Watch ...,Automatic,Steel,26 mm,2014.0,Very good,"Original box, original papers","United States of America, Georgia"
1,Rolex Chronograph,2917,REF. 2917,Manual winding,Steel,33 mm,1934.0,Very good,"Original box, no original papers","Italy, Roma"
2,Rolex Daytona,116519G,保証書付き ROLEX ロレックス デイトナ コスモグラフ K18WG 8Pダイヤ ランダム...,Automatic,White gold,39 mm,NaN,Fair,"Original papers, no original box","Japan, Nagoya City"
3,Rolex Submariner Date,116613,New Submariner 116613 Yellow Steel Gold Cerami...,Automatic,Steel,40 mm,2020.0,Unworn,"Original box, original papers","United States of America, Florida, Miami"
4,Rolex Submariner Date,16610,1990s ROLEX SUBMARINER 16610 Tritium vintage G...,Automatic,NaN,40 mm,1990.0,Good,"Original box, original papers","Japan, Ehime yawatahama"


Percentage of missing values

In [41]:
cate_missing_percentages = (categorical_cols.isna()).sum() / rolex_df.shape[0] * 100
cate_missing_percentages

model                  0.000000
reference number       3.207654
ad name                0.088931
movement               3.624711
case material          5.363468
case diameter          4.457290
year of production    25.888161
condition              1.407565
scope of delivery      0.000000
location               0.000000
dtype: float64

Number of different values

In [42]:
categorical_cols.nunique()

model                    58
reference number       4863
ad name               44292
movement                  3
case material            13
case diameter           643
year of production      111
condition                 7
scope of delivery         4
location               4097
dtype: int64

---

# Asking meaningful questions 

In [43]:
rolex_df.drop(columns = ['ad name'],inplace=True)
rolex_df = rolex_df[rolex_df['model'] != 'Rolex']

In [44]:
# #Remove watch's size in model name
# rolex_df['model'].replace(regex = True,to_replace = r"[0-9]",value = '',inplace= True)

# # #Get true size of case
# rolex_df['case diameter'] = rolex_df['case diameter'].str.extract(r'(^[\d][\d])')

# #Preprocess ref number because some ref num are in wrong format (including characters,etc) 

# tmp = rolex_df['reference number'].str.extract(r'(\d+[-]\d+)|(\d+)')
# tmp[0].fillna(tmp[1],inplace=True)
# rolex_df['reference number'] = tmp[0]

# #Replace all Nan with Unknown
# rolex_df['reference number'].replace(regex = True,to_replace = "",value = 'Unknown',inplace= True)

In [45]:
rolex_df.reset_index(drop=True,inplace = True)
rolex_df.head()

,model,reference number,price,aditional shipping price,movement,case material,case diameter,year of production,condition,scope of delivery,location
0,Rolex Lady-Datejust,179161,9080.0,140.0,Automatic,Steel,26 mm,2014.0,Very good,"Original box, original papers","United States of America, Georgia"
1,Rolex Chronograph,2917,16202.0,216.0,Manual winding,Steel,33 mm,1934.0,Very good,"Original box, no original papers","Italy, Roma"
2,Rolex Daytona,116519G,41567.0,0.0,Automatic,White gold,39 mm,NaN,Fair,"Original papers, no original box","Japan, Nagoya City"
3,Rolex Submariner Date,116613,19795.0,235.0,Automatic,Steel,40 mm,2020.0,Unworn,"Original box, original papers","United States of America, Florida, Miami"
4,Rolex Submariner Date,16610,10674.0,145.0,Automatic,NaN,40 mm,1990.0,Good,"Original box, original papers","Japan, Ehime yawatahama"


Fill ref num movement case material and year of production with the mode of their model.  

In [46]:
# rolex_df['year of production'] = rolex_df['year of production'].astype(np.number)
# rolex_df.loc[rolex_df['year of production'] < 1905, 'year of production' ] = np.nan
# rolex_df['year of production'] = rolex_df['year of production'].astype('object')

# model_list = rolex_df['model'].unique()
# for i in model_list:
#     rolex_df[rolex_df['model'] == i] = rolex_df[rolex_df['model'] == i].fillna(rolex_df[rolex_df['model'] == i].mode().iloc[0])


Check if dataset still consists missing values

In [47]:
missing_percentages = rolex_df.isna().sum()/rolex_df.shape[0]*100
missing_percentages

model                        0.000000
reference number             2.672060
price                        6.321740
aditional shipping price     0.000000
movement                     3.574719
case material                5.241048
case diameter                4.289975
year of production          25.630534
condition                    1.361798
scope of delivery            0.000000
location                     0.000000
dtype: float64

### Question 1 : How can we know if we had bought a great deal (Our team will create a model to predict price of watch) 
Have you ever wanted to buy a watch but you do not know that the price of the thing that you found is cheap or expensive. That is the reason why we want to ask this question is that our team was in this situations multiple times. We find that it is really time consuming when you have to find the things which we want to buy at a good price. Therefore, we will create a model that can help us and other people who want to buy Rolex save time from searching for a great deal. And if we can solve this problem, we can use this model to predict price of various things not only Rolex watches.

In [48]:
class preprocess(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
        
    def transform (self, X_df):
        out_df = X_df.copy()
    
        #Remove watch's size in model name
        out_df['model'].replace(regex = True,to_replace = r"[0-9]",value = '',inplace= True)

        # #Get true size of case
        out_df['case diameter'] = out_df['case diameter'].str.extract(r'(^[\d][\d])')

        #Preprocess ref number because some ref num are in wrong format (including characters,etc) 

        tmp = out_df['reference number'].str.extract(r'(\d+[-]\d+)|(\d+)')
        tmp[0].fillna(tmp[1],inplace=True)
        out_df['reference number'] = tmp[0]

        #Replace all Nan with Unknown
        out_df['reference number'].replace(regex = True,to_replace = "",value = 'Unknown',inplace= True)

        out_df['year of production'] = out_df['year of production'].astype(np.number)
        out_df.loc[out_df['year of production'] < 1905, 'year of production' ] = np.nan
        out_df['year of production'] = out_df['year of production'].astype('object')

        model_list = out_df['model'].unique()
        for i in model_list:
            out_df[out_df['model'] == i] = out_df[out_df['model'] == i].fillna(out_df[out_df['model'] == i].mode().iloc[0])
        
        return out_df

Split data

In [49]:
# X_df = rolex_df.drop(columns=['price'])
# y_df = rolex_df['price']
tmp  = rolex_df[['model', 'reference number', 'price',
       'movement', 'case material', 'case diameter', 'year of production',
       'condition', 'scope of delivery']].copy()

X_train, X_test= train_test_split(tmp, test_size=0.2, random_state=0)
X_train = preprocess().transform(X_train)
Y_train = X_train['price']

X_test = preprocess().transform(X_test)
Y_test = X_test['price']

In [50]:
preprocess_data =  preprocess().transform(X_train)

In [51]:
categorical_cols = preprocess_data.select_dtypes(exclude=np.number).columns
numerical_cols = preprocess_data.select_dtypes(include = np.number).columns

ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
scaler = StandardScaler()

numerics_pipeline = make_pipeline(scaler)
categorical_pipeline = make_pipeline(ohe,scaler)

col_transformer = make_column_transformer(
    (numerics_pipeline,numerical_cols),
    (categorical_pipeline,categorical_cols),
    remainder='passthrough')

preprocess_pipeline = make_pipeline(preprocess(), col_transformer)
# preprocessed_train_X = preprocess_pipeline.fit_transform(X_train)

Predict

In [52]:
full_pipeline = make_pipeline(preprocess_pipeline,RandomForestRegressor(n_jobs = -1,verbose = True))

# Experiment with different values of hyperparameters (the whole process can take 2-3 minutes) and choose the best values
# val_accs = []
# for alpha in alphas:
#     for num_top_cp_values in num_top_cp_values_s:
     

clf = full_pipeline.fit(X_train,Y_train)

# val_accs.append(clf.score(X_test,Y_test)*100)

# val_accs
predict_y = clf.predict(X_test)
# clf.shape

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [53]:
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt
sqrt(mean_squared_error(predict_y, Y_test))

314.8829677336264

In [54]:
r2_score(Y_test,predict_y)


0.9998922421992859

In [55]:
clf.score(X_test,Y_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


0.9998922421992859

In [56]:
clf.score(X_train,Y_train)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.2s finished


0.9998497337837975